In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
spec_data = [
    (pd.read_csv("fix_flags.bc.20180813-135517.false.csv"), 'false'),
    (pd.read_csv("fix_flags.bc.20180812-142333.spread.csv"), 'spread'),
    (pd.read_csv("fix_flags.bc.20180813-025019.close.csv"), 'close'),
    (pd.read_csv("fix_flags.bc.20180814-034312.true.csv"), 'true')
]
npb_data = [
    (pd.read_csv("fix_flags.bc.20180813-124212.false.csv"), 'false'),
    (pd.read_csv("fix_flags.bc.20180812-142221.spread.csv"), 'spread'),
    (pd.read_csv("fix_flags.bc.20180812-204259.close.csv"), 'close'),
    (pd.read_csv("fix_flags.bc.20180814-034012.true.csv"), 'true')
]

In [ ]:
flags = ['O1', 'O2', 'O3']
markers = ['x', 'o', 'v']

def compare_results(frames):
    benchmarks = frames[0][0]["Benchmark"].unique()
    
    rows = len(benchmarks)
    columns = len(frames)

    fig = plt.figure(figsize=(8.0*columns, 6.0*rows))
    
    benchmark_axes = {}
    
    for i, (frame, label) in enumerate(frames):
        frame.loc[:, 'Energy'] *= 1e-6 # Convert energy to Joules
        
        plot_pos = i + 1
        
        for benchmark in benchmarks:
            benchmark_data = frame[frame["Benchmark"] == benchmark]
            
            if benchmark in benchmark_axes:
                ax = fig.add_subplot(rows, columns, plot_pos, sharex=benchmark_axes[benchmark], sharey=benchmark_axes[benchmark])
            else:
                ax = fig.add_subplot(rows, columns, plot_pos)
                benchmark_axes[benchmark] = ax
                
            plot_pos += columns
            
            for k, flag in enumerate(flags):

                marker = markers[k]

                flag_data = benchmark_data[benchmark_data["Flags"] == "-" + flag]

                x = flag_data["Time"].values
                y = flag_data["Energy"].values

                ax.scatter(x, y, marker=marker, label=flag)
            
            ax.legend()
            ax.set_title(benchmark + ', proc_bind=' + label)


In [ ]:
compare_results(spec_data)

In [ ]:
compare_results(npb_data)

In [ ]:
def plot_benchmark_results(benchmark, frame):
    data = frame[frame["Benchmark"] == benchmark]
    
    flags = ['O1', 'O2', 'O3']
    markers = ['x', 'o', 'v']
    
    fig, ax = plt.subplots()
    for i in range(len(flags)):
        flag = flags[i]
        marker = markers[i]
        
        flag_data = data[data["Flags"] == "-" + flag]
        
        x = flag_data["Time"].values
        y = flag_data["Energy"].values
        
        ax.scatter(x, y, marker=marker, label=flag)

    plt.legend()        
    plt.xlabel('Time (s)')
    plt.ylabel('Energy (J)')
    plt.title('Energy vs Time for ' + benchmark + ' benchmark')

def summary(data, variable):
    mean = np.mean(data[variable].values)
    median = np.median(data[variable].values)
    min = np.min(data[variable].values)
    max = np.max(data[variable].values)
    range = max - min
    relative_range = range / mean * 100
    std = np.std(data[variable].values)
    coeff_var = std / mean * 100
    
    return [mean, median, min, max, range, relative_range, std, coeff_var]

def summary_frame(array):
    return pd.DataFrame(array[:,1:], 
                       index=array[:,0],
                       dtype=float,
                       columns=['Mean', 'Median', 'Min', 'Max', 'Range', 'Relative Range', 'Std', 'CV'])